# How to build a simple Chatbot with stored memory using LangChain

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [3]:
# connect with .env file 
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
# Connect with an LLM and start a conversation with it

from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

In [6]:
# Call the ChatModel (the LLM)¶
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is a calming and relaxing color that reminds me of the ocean and the sky. It is a versatile color that can be both soothing and invigorating. I love how it can be both subtle and bold, depending on the shade. Blue is a timeless and classic color that never goes out of style. It's a color that brings me a sense of peace and tranquility whenever I see it.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 13, 'total_tokens': 94}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d81b27e-e61d-4479-9a19-4a5f2d9df493-0', usage_metadata={'input_tokens': 13, 'output_tokens': 81, 'total_tokens': 94})

In [7]:
# Check if the Chatbot remembers your favorite color.¶
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have the ability to know your favorite color.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 13, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-48c63b6b-c591-4e52-8970-ff0e58e0aabe-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29})

## Let's add memory to our Chatbot

In [10]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [11]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [12]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [13]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [14]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [15]:
chain.invoke("my name is Julio")

{'content': 'my name is Julio',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': "Hello Julio! It's nice to meet you. How can I help you today?"}

In [16]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is Julio'),
  AIMessage(content="Hello Julio! It's nice to meet you. How can I help you today?")],
 'text': 'Your name is Julio.'}